# Generative AI - Notebook of the Project

Team members: Binesh Arakkal Remesh, Zahra Sharifi, Elise Soenen

## 1. Installations

In [ ]:
! pip install transformers[torch] datasets==3.6.0 --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 13.7 MB/s eta 0:00:00


In [ ]:
! pip install ctranslate2 evaluate sacrebleu --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.8/38.8 MB 27.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 9.7 MB/s eta 0:00:00


In [ ]:
import torch
import transformers

In [ ]:
import json
from tqdm.notebook import tqdm
#import evaluate

In [ ]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
tokenizer = AutoTokenizer.from_pretrained("google-t5/t5-small")
model = AutoModelForSeq2SeqLM.from_pretrained("google-t5/t5-small")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [ ]:
from datasets import load_dataset
data = load_dataset('GEM/e2e_nlg', trust_remote_code=True)

README.md: 0.00B [00:00, ?B/s]

e2e_nlg.py: 0.00B [00:00, ?B/s]

dataset_infos.json: 0.00B [00:00, ?B/s]

Generating train split:   0%|          | 0/33525 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1484 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1847 [00:00<?, ? examples/s]

Generating challenge_train_sample split:   0%|          | 0/500 [00:00<?, ? examples/s]

Generating challenge_validation_sample split:   0%|          | 0/500 [00:00<?, ? examples/s]

Generating challenge_test_scramble split:   0%|          | 0/500 [00:00<?, ? examples/s]

In [ ]:
data['train'][55]

{'gem_id': 'e2e_nlg-train-55',
 'gem_parent_id': 'e2e_nlg-train-55',
 'meaning_representation': 'name[The Golden Curry], food[Italian], customer rating[5 out of 5], area[city centre], familyFriendly[no], near[Café Rouge]',
 'target': 'The Golden Curry is near Café Rouge with will serve you with some lovely Italian food with in the city center with a rating of 5 out of 5 stars. no children please.',
 'references': []}

In [ ]:
data['validation'][50]

{'gem_id': 'e2e_nlg-validation-50',
 'gem_parent_id': 'e2e_nlg-validation-50',
 'meaning_representation': 'name[Aromi], eatType[coffee shop], food[Chinese], customer rating[high], area[riverside]',
 'target': 'Aromi is a coffee shop providing Chinese food It is located in the riverside. Its customer rating is high.',
 'references': ['Aromi is a coffee shop providing Chinese food It is located in the riverside. Its customer rating is high.',
  'Aromi is a coffee shop providing Chinese food It is located in the riverside. Its customer rating is high.']}

In [ ]:
data['test'][2]

{'gem_id': 'e2e_nlg-test-2',
 'gem_parent_id': 'e2e_nlg-test-2',
 'meaning_representation': 'eatType[restaurant], food[Chinese], priceRange[high], customer rating[1 out of 5], area[riverside], familyFriendly[no], near[All Bar One]',
 'target': 'In the riverside area near the All Bar One there is a non child friendly restaurant. The Cricketerss serves Chinese food in a high price range and has a customer rating of 1 out of 5.',
 'references': ['In the riverside area near the All Bar One there is a non child friendly restaurant. The Cricketerss serves Chinese food in a high price range and has a customer rating of 1 out of 5.']}

## 2. Preprocessing

In [ ]:
def linearize_mr(mr: str, add_task_prefix: bool = True) -> str:
    """
    Convert an E2E NLG MR string into a linearized text format.
    """


    slot_value_pairs = [pair.strip() for pair in mr.split(",")]  # Split by commas → slot-value pairs


    formatted_pairs = []    # Convert each  values to  "slot = value"
    for pair in slot_value_pairs:
        if "[" in pair and "]" in pair:
            slot, value = pair.split("[", 1)
            slot = slot.strip()
            value = value.strip(" ]")  # remove ] and spaces
            formatted_pairs.append(f"{slot} = {value}")
        else:
            formatted_pairs.append(pair)  # fallback if formatting is odd


    linearized = " ; ".join(formatted_pairs)   # Join pairs with "; "

    if add_task_prefix:
        return f"translate MR to text: {linearized}"
    else:
        return linearized

#print(linearize_mr(mr))

In [ ]:
def simplify_train_dataset_singl_references(batch):
    new_batch = {
        'graph':[],
        'input':[],
        'reference':[]
    }
    for i in range(len(batch['meaning_representation'])):  #our target dataset is 'meaning_representation'
        graph = batch['meaning_representation'][i]
        new_batch['graph'].append(graph)
        # Create an input string from the graph data
        new_batch['input'].append(linearize_mr(graph))
        new_batch['reference'].append(batch['target'][i])
    return new_batch

original_columns = data['test'].features
train_ds = data['train'].map(simplify_train_dataset_singl_references, batched=True, remove_columns=original_columns)

Map:   0%|          | 0/33525 [00:00<?, ? examples/s]

In [ ]:
train_ds[50]

{'graph': 'name[Cocum], eatType[coffee shop], food[Fast food], priceRange[cheap], familyFriendly[yes]',
 'input': 'translate MR to text: name = Cocum ; eatType = coffee shop ; food = Fast food ; priceRange = cheap ; familyFriendly = yes',
 'reference': 'Cocum for coffee shop offers Fast food for cheap price for kids Friendly.'}

In [ ]:
def simplify_validation_dataset_sinlge_references(batch):
    new_batch = {
        'graph':[],
        'input':[],
        'reference':[]
    }
    for i in range(len(batch['meaning_representation'])):  #our target dataset is 'meaning_representation'
        graph = batch['meaning_representation'][i]
        new_batch['graph'].append(graph)
        new_batch['input'].append(linearize_mr(graph))
        new_batch['reference'].append(batch['references'][i][0])
    return new_batch

In [ ]:
def simplify_test_dataset_sinlge_references(batch):
    new_batch = {
        'graph':[],
        'input':[],
        'reference':[]
    }
    for i in range(len(batch['meaning_representation'])):  #our target dataset is 'tripleset'
        graph = batch['meaning_representation'][i]
        new_batch['graph'].append(graph)
        new_batch['input'].append(linearize_mr(graph))
        new_batch['reference'].append(batch['references'][i])
    return new_batch

In [ ]:
test_ds = data['test'].map(simplify_test_dataset_sinlge_references, batched=True, remove_columns=original_columns)

Map:   0%|          | 0/1847 [00:00<?, ? examples/s]

In [ ]:
valid_ds = data['validation'].map(simplify_validation_dataset_sinlge_references, batched=True, remove_columns=original_columns)

Map:   0%|          | 0/1484 [00:00<?, ? examples/s]

In [ ]:
train_ds[60]

{'graph': 'name[The Cambridge Blue], eatType[restaurant], food[Chinese], customer rating[high]',
 'input': 'translate MR to text: name = The Cambridge Blue ; eatType = restaurant ; food = Chinese ; customer rating = high',
 'reference': 'The Cambridge Blue is a restaurant that provides Chinese food Its customer rating is high.'}

In [ ]:
valid_ds[50]

{'graph': 'name[Aromi], eatType[coffee shop], food[Chinese], customer rating[high], area[riverside]',
 'input': 'translate MR to text: name = Aromi ; eatType = coffee shop ; food = Chinese ; customer rating = high ; area = riverside',
 'reference': 'Aromi is a coffee shop providing Chinese food It is located in the riverside. Its customer rating is high.'}

In [ ]:
test_ds[0]

{'graph': 'eatType[pub], food[Fast food], customer rating[high], area[riverside], familyFriendly[no], near[Café Rouge]',
 'input': 'translate MR to text: eatType = pub ; food = Fast food ; customer rating = high ; area = riverside ; familyFriendly = no ; near = Café Rouge',
 'reference': ['The Mills is not kid friendly as it is a riverside pub near Café Rouge.  Its mid priced fast food is highly rated.']}

## 3. Testing the model without fine-tuning for comparison

In [ ]:

# Ensure model is on the right device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Define generation parameters
MAX_INPUT_LENGTH = 128
MAX_OUTPUT_LENGTH = 128
BATCH_SIZE = 16  # you can increase if GPU memory allows

# Prepare batches
inputs = test_ds["input"]
results = []

print(f"Running generation on {len(inputs)} test samples...")

for i in tqdm(range(0, len(inputs), BATCH_SIZE)):
    batch_inputs = inputs[i : i + BATCH_SIZE]

    # Tokenize
    encoded = tokenizer(
        batch_inputs,
        return_tensors="pt",
        padding=True,
        truncation=True,
        max_length=MAX_INPUT_LENGTH
    ).to(device)

    # Generate
    with torch.no_grad():
        outputs = model.generate(
            **encoded,
            max_length=MAX_OUTPUT_LENGTH,
            num_beams=4,
            early_stopping=True
        )

    # Decode
    decoded_outputs = tokenizer.batch_decode(outputs, skip_special_tokens=True)

    # Append to results
    for j, output_text in enumerate(decoded_outputs):
        results.append({
            "graph": test_ds[i + j]["graph"],
            "input": test_ds[i + j]["input"],
            "reference": test_ds[i + j]["reference"],
            "t5-small_generated": output_text
        })

# Save results to JSON
output_file = "t5_small_pre_finetune_results.json"
with open(output_file, "w", encoding="utf-8") as f:
    json.dump(results, f, indent=4, ensure_ascii=False)

print(f"\n✅ Results saved to: {output_file}")


Running generation on 1847 test samples...


  0%|          | 0/116 [00:00<?, ?it/s]

## 4. Training

In [ ]:
# Define function to tokenize dataset:
max_length = 10
def tokenize_(sample):
    tokenized_sample = tokenizer(sample['input'], text_target=sample['reference'], padding=True)
    tokenized_sample['labels'] = tokenized_sample['labels'] # Rename 'reference' to 'labels'
    return tokenized_sample

In [ ]:
tokenized_train_ds = train_ds.map(tokenize_, remove_columns=['input', 'graph', 'reference'], batched=True, batch_size=2000)
tokenized_valid_ds = valid_ds.map(tokenize_, remove_columns=['input', 'graph', 'reference'], batched=True, batch_size=2000)

In [ ]:
training_arguments = transformers.Seq2SeqTrainingArguments(
    output_dir = 't5-e2e_nlg',
    num_train_epochs = 3,
    per_device_train_batch_size = 16,
    per_device_eval_batch_size = 32,
    gradient_accumulation_steps = 2,
    eval_strategy = 'steps',
    eval_steps = 500,
    save_total_limit = 3,
    load_best_model_at_end = True,
    bf16=True,
    optim='adafactor',
    report_to = []
)

In [ ]:
trainer = transformers.Seq2SeqTrainer(
    model= model,
    args=training_arguments,
    train_dataset= tokenized_train_ds,
    eval_dataset= tokenized_valid_ds,
    tokenizer= tokenizer,
    data_collator=transformers.DataCollatorForSeq2Seq(tokenizer= tokenizer)
        )

/tmp/ipython-input-1112203840.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = transformers.Seq2SeqTrainer(


In [ ]:
trainer.train()

Step,Training Loss,Validation Loss
500,0.929300,0.471848
1000,0.581100,0.441044
1500,0.549900,0.428667
2000,0.537500,0.421409
2500,0.527400,0.417099
3000,0.522400,0.415591


There were missing keys in the checkpoint model loaded: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight', 'lm_head.weight'].


TrainOutput(global_step=3144, training_loss=0.6043817845312996, metrics={'train_runtime': 1693.0887, 'train_samples_per_second': 59.403, 'train_steps_per_second': 1.857, 'total_flos': 2048633427394560.0, 'train_loss': 0.6043817845312996, 'epoch': 3.0})

In [ ]:
test_ds[0]

{'graph': 'eatType[pub], food[Fast food], customer rating[high], area[riverside], familyFriendly[no], near[Café Rouge]',
 'input': 'translate MR to text: eatType = pub ; food = Fast food ; customer rating = high ; area = riverside ; familyFriendly = no ; near = Café Rouge',
 'reference': ['The Mills is not kid friendly as it is a riverside pub near Café Rouge.  Its mid priced fast food is highly rated.']}

In [ ]:
trainer.model.save_pretrained('t5-e2e_nlg')
tokenizer.save_pretrained('t5-e2e_nlg')

('t5-e2e_nlg/tokenizer_config.json',
 't5-e2e_nlg/special_tokens_map.json',
 't5-e2e_nlg/spiece.model',
 't5-e2e_nlg/added_tokens.json',
 't5-e2e_nlg/tokenizer.json')

In [ ]:
input_text = test_ds[25]['input']
first_reference = test_ds[25]['reference']

# Tokenize the input text
input_ids = tokenizer(input_text, return_tensors="pt").input_ids.cuda() # Move to GPU if model is on GPU

# Generate output using the model
output_tokens = model.generate(input_ids, max_length=500) # Specify max_length for generation

# Decode the generated tokens back to text
output_text = tokenizer.decode(output_tokens[0], skip_special_tokens=True)

print(f'Input text: {input_text}')
print()
print(f'First reference: {first_reference}')
print()
print(f'Output text: {output_text}')

## 5. Testing

In [ ]:
# Convert the model to ctranslate format
! ct2-transformers-converter --model ./t5-e2e_nlg --output_dir t5-small-e2e_nlg-ct2

Traceback (most recent call last):
  File "/usr/local/bin/ct2-transformers-converter", line 8, in <module>
    sys.exit(main())
             ^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/ctranslate2/converters/transformers.py", line 2889, in main
    converter.convert_from_args(args)
  File "/usr/local/lib/python3.12/dist-packages/ctranslate2/converters/converter.py", line 50, in convert_from_args
    return self.convert(
           ^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/ctranslate2/converters/converter.py", line 84, in convert
    raise RuntimeError(
RuntimeError: output directory t5-small-e2e_nlg-ct2 already exists, use --force to override


In [ ]:
# Instantiate model as ctranslate Translator and instantiate TOkenizer
import ctranslate2
translator = ctranslate2.Translator(
    't5-small-e2e_nlg-ct2',
    device='cuda',
)
tokenizer = transformers.AutoTokenizer.from_pretrained('./t5-e2e_nlg')

In [ ]:
# Define function to pre process inputs
def pre_process(text):
    input_ids = tokenizer.encode(text)
    input_tokens = tokenizer.convert_ids_to_tokens(input_ids)
    return input_tokens

In [ ]:
def post_process(output):
    output_tokens = output.hypotheses[0]
    output_ids = tokenizer.convert_tokens_to_ids(output_tokens)
    output_text = tokenizer.decode(output_ids, skip_special_tokens=True)
    return output_text

In [ ]:
batch = [pre_process(text) for text in tqdm(test_ds['input'])]
batch_size = 32

generated_texts = []
for i in tqdm(range(0, len(batch), batch_size)):
    subbatch = batch[i : i + batch_size]
    output = translator.translate_batch(subbatch, max_batch_size=batch_size)
    generated_texts += [post_process(o) for o in output]

final_data = test_ds.add_column('t5-small_e2e_nlg', generated_texts)
final_data[0]

  0%|          | 0/1847 [00:00<?, ?it/s]

  0%|          | 0/58 [00:00<?, ?it/s]

{'graph': 'eatType[pub], food[Fast food], customer rating[high], area[riverside], familyFriendly[no], near[Café Rouge]',
 'input': 'translate MR to text: eatType = pub ; food = Fast food ; customer rating = high ; area = riverside ; familyFriendly = no ; near = Café Rouge',
 'reference': ['The Mills is not kid friendly as it is a riverside pub near Café Rouge.  Its mid priced fast food is highly rated.'],
 't5-small_e2e_nlg': 'Near Café Rouge in the riverside area is a pub that serves fast food with a high customer rating. It is not family-friendly.'}

In [ ]:
final_data.to_json('t5_small_e2e_nlg_gen-response.json')

Creating json from Arrow format:   0%|          | 0/2 [00:00<?, ?ba/s]

1530999

## 6. Evaluation

In [ ]:
from sacrebleu import CHRF
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt

In [ ]:
evaluate_chrfpp = evaluate.load('chrf')

In [ ]:
ds = load_dataset('json', data_files='t5_small_e2e_nlg_gen-response.json')

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
ds_pre_tune = load_dataset('json', data_files='t5_small_pre_finetune_results.json')

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
ds

DatasetDict({
    train: Dataset({
        features: ['graph', 'input', 'reference', 't5-small_e2e_nlg'],
        num_rows: 1847
    })
})

In [ ]:
ds_pre_tune

DatasetDict({
    train: Dataset({
        features: ['graph', 'input', 'reference', 't5-small_generated'],
        num_rows: 1847
    })
})

In [ ]:
final_dataset = ds['train']

In [ ]:
final_pre_data = ds_pre_tune['train']

In [ ]:
max_references_pre = max([len(gen) for gen in ds_pre_tune['train']['reference']])
max_references_pre

35

In [ ]:
max_references = max([len(gen) for gen in ds['train']['reference']])
max_references

35

In [ ]:
padded_references = []
for references in final_pre_data['reference']:
    temp = references.copy()
    while len(temp) < max_references_pre:
        temp.append('')
    padded_references.append(temp)

In [ ]:
evaluate_chrfpp_score = evaluate_chrfpp.compute(predictions=final_pre_data['t5-small_generated'], references=padded_references, word_order=2)
evaluate_chrfpp_score

{'score': 41.5957888209607, 'char_order': 6, 'word_order': 2, 'beta': 2}

In [ ]:
!pip install rouge_score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=2296835358a9e30bad06a33ded184bfd7b8c804782fa387d8afc077fdb64d0ff
  Stored in directory: /root/.cache/pip/wheels/85/9d/af/01feefbe7d55ef5468796f0c68225b6788e85d9d0a281e7a70
Successfully built rouge_score


In [ ]:
bleu = evaluate.load("bleu")
meteor = evaluate.load("meteor")
rouge = evaluate.load("rouge")

results_bleu = bleu.compute(predictions=final_pre_data['t5-small_generated'], references=padded_references)
results_meteor = meteor.compute(predictions=final_pre_data['t5-small_generated'], references=padded_references)
results_rouge = rouge.compute(predictions=final_pre_data['t5-small_generated'], references=padded_references)

print("BLEU:", results_bleu)
print("METEOR:", results_meteor)
print("ROUGE:", results_rouge)


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


BLEU: {'bleu': 0.06262498552757835, 'precisions': [0.34546320005252446, 0.12925504003249996, 0.04296697536257208, 0.008016900492931044], 'brevity_penalty': 1.0, 'length_ratio': 1903.875, 'translation_length': 60924, 'reference_length': 32}
METEOR: {'meteor': np.float64(0.3680615816544237)}
ROUGE: {'rouge1': np.float64(0.5071012194877771), 'rouge2': np.float64(0.23882645546505105), 'rougeL': np.float64(0.37579867735240025), 'rougeLsum': np.float64(0.37580553322044674)}


In [ ]:
ds_pre_tune['train'][555]

{'graph': 'name[The Mill], eatType[pub], priceRange[cheap], customer rating[high], area[riverside], familyFriendly[no], near[Café Rouge]',
 'input': 'translate MR to text: name = The Mill ; eatType = pub ; priceRange = cheap ; customer rating = high ; area = riverside ; familyFriendly = no ; near = Café Rouge',
 'reference': ['In the riverside, near Café Rouge, there is The Mill, a cheap highly rated non-kids friendly pub.',
  'The Mill is a cheap highly rated non-kids friendly pub, located in the riverside, near Café Rouge.'],
 't5-small_generated': 'MR to text: name = The Mill ; eatType = pub ; priceRange = cheap ; customer rating = high ; area = riverside ; familyFriendly = no ; near = Café Rouge'}

In [ ]:
data = load_dataset('json', data_files='t5_small_e2e_nlg_gen-response (2).json')

In [ ]:
data['train'][26]

{'graph': 'name[Blue Spice], eatType[pub], food[English], area[riverside], near[Rainbow Vegetarian Café]',
 'input': 'translate MR to text: name = Blue Spice ; eatType = pub ; food = English ; area = riverside ; near = Rainbow Vegetarian Café',
 'reference': ['Situated near the Rainbow Vegetarian Café in the riverside area of the city, The Blue Spice pub, is ideal if you fancy traditional English food whilst out with the kids.'],
 't5-small_e2e_nlg': 'Blue Spice is a pub that serves English food in the riverside area near Rainbow Vegetarian Café.'}

In [ ]:
data['train'][555]

{'graph': 'name[The Mill], eatType[pub], priceRange[cheap], customer rating[high], area[riverside], familyFriendly[no], near[Café Rouge]',
 'input': 'translate MR to text: name = The Mill ; eatType = pub ; priceRange = cheap ; customer rating = high ; area = riverside ; familyFriendly = no ; near = Café Rouge',
 'reference': ['In the riverside, near Café Rouge, there is The Mill, a cheap highly rated non-kids friendly pub.',
  'The Mill is a cheap highly rated non-kids friendly pub, located in the riverside, near Café Rouge.'],
 't5-small_e2e_nlg': 'The Mill is a cheap pub near Café Rouge in the riverside area. It is not family-friendly and has a high customer rating.'}

In [ ]:
data['train'][500]

{'graph': 'name[The Mill], eatType[pub], food[Fast food], priceRange[cheap], customer rating[average], area[city centre], familyFriendly[yes], near[Café Sicilia]',
 'input': 'translate MR to text: name = The Mill ; eatType = pub ; food = Fast food ; priceRange = cheap ; customer rating = average ; area = city centre ; familyFriendly = yes ; near = Café Sicilia',
 'reference': ['The fast food place with an average customer rating, The Mill, is near Café Sicilia in the city center, has a cheap price range and is a family-friendly pub.'],
 't5-small_e2e_nlg': 'The Mill is a cheap, family friendly fast food pub located in the city centre near Café Sicilia. It has an average customer rating.'}